In [1]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.5.0
torchvision version: 0.20.0


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular.going_modular import data_setup, engine
except:
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data'
directories = ['processed_lsd_jpgs', 'processed_osf_jpgs', 'processed_spider_jpgs', 'processed_tseg_jpgs']

train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

for d in directories:
    os.makedirs(os.path.join(train_dir, d), exist_ok=True)
    os.makedirs(os.path.join(test_dir, d), exist_ok=True)

for d in directories:
    image_dir = os.path.join(dataset_path, d)
    images = os.listdir(image_dir)

    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_images:
        src_path = os.path.join(image_dir, img)
        dest_path = os.path.join(train_dir, d, img)  
        shutil.copy(src_path, dest_path)

    for img in test_images:
        src_path = os.path.join(image_dir, img)
        dest_path = os.path.join(test_dir, d, img) 
        shutil.copy(src_path, dest_path)

print("Train-test split completed.")


Train-test split completed.


In [7]:
import os

def count_images_in_directory(directory):
    # Filter files by common image extensions
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')
    images = [img for img in os.listdir(directory) if img.endswith(image_extensions)]
    
    return len(images)

# Specify the directory you want to check
directory = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/train/random'  # Replace with your directory path

image_count = count_images_in_directory(directory)
print(f"Number of images in the directory '{directory}': {image_count}")


Number of images in the directory '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/train/random': 867


In [8]:
data_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data'


In [9]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [10]:
train_dir = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/train'
test_dir = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/test'

In [11]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS):

  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  class_names = train_data.classes

  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )
  return train_datalaoder, test_dataloader, class_names


In [12]:
manual_transforms = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    normalize
]) # can also use automatic transforms using weights.transforms()

train_dataloaders, test_dataloaders, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=32
)

train_dataloaders, test_dataloaders, class_names

(<torch.utils.data.dataloader.DataLoader at 0x31a137750>,
 ['processed_lsd_jpgs',
  'processed_osf_jpgs',
  'processed_spider_jpgs',
  'processed_tseg_jpgs',
  'random'])

In [13]:
class_names

['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs',
 'random']

In [14]:
import ssl
import urllib.request

# Create an SSL context that doesn't verify certificates
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Make your request with the modified context
url = "https://example.com"
response = urllib.request.urlopen(url, context=ssl_context)


In [15]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load the EfficientNetV2_M model without pre-trained weights
model = models.efficientnet_v2_m(weights=None)

# Path to your .pth file (replace with actual path)
pth_file = 'efficientnet_v2_m-dc08266a.pth'

# Load the saved weights from the .pth file
checkpoint = torch.load(pth_file)

# Load the weights into the model
model.load_state_dict(checkpoint)

/var/folders/4x/9705y5ws5p31tnxd9bmg3ll00000gn/T/ipykernel_19078/1801364724.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pth_file)


<All keys matched successfully>

In [16]:
for param in model.features.parameters():
    param.requires_grad = False
    

set_seeds() 

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names),
              bias=True).to(device))

In [28]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.models.efficientnet as efficientnet

model = models.efficientnet_v2_m(weights=None)

for param in model.features.parameters():
    param.requires_grad = False

num_classes = len(class_names)  # Assuming you have defined `class_names`
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=num_classes, bias=True)
)

# Move the model to the device (e.g., GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# (Optional) Load weights if you have a trained model already
# Load saved weights from the .pth file (if you want to fine-tune or continue training)
model.load_state_dict(torch.load('new_trained_model_msa.pth', map_location=device))

# Step 4: Save the model after fine-tuning or training

print('Model saved successfully!')


Model saved successfully!


/var/folders/4x/9705y5ws5p31tnxd9bmg3ll00000gn/T/ipykernel_19078/798647838.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('new_trained

In [17]:
from torchinfo import summary


In [18]:
summary(model, 
        input_size=(32, 3, 224, 224), 
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 5]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 24, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 24, 112, 112]   (648)                False
│    │    └─BatchNorm2d (1)                                  [32, 24, 112, 112]   [32, 24, 112, 112]   (48)                 False
│    │    └─SiLU (2)                                         [32, 24, 112, 112]   [32, 24, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 24, 112, 112]   [32, 

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    print("[INFO] Couldn't find tensorboard... installing it.")
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter()

In [21]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
        
        writer.add_graph(model=model, 
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))
    
    writer.close()
    
    return results

In [22]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloaders,
                test_dataloader=test_dataloaders,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=3,
                device=device)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.7439 | train_acc: 0.8351 | test_loss: 0.3806 | test_acc: 0.9219


 33%|███▎      | 1/3 [16:04<32:09, 964.94s/it]

Epoch: 2 | train_loss: 0.3187 | train_acc: 0.9294 | test_loss: 0.2768 | test_acc: 0.9241


 67%|██████▋   | 2/3 [32:59<16:34, 994.20s/it]

Epoch: 3 | train_loss: 0.2555 | train_acc: 0.9353 | test_loss: 0.2317 | test_acc: 0.9286


100%|██████████| 3/3 [51:52<00:00, 1037.44s/it]


In [24]:
results

{'train_loss': [0.7438545823097229, 0.31871131011124315, 0.2555431628535534],
 'train_acc': [0.8351293103448276, 0.9294181034482759, 0.9353448275862069],
 'test_loss': [0.3806075932724135, 0.2767751520233495, 0.23165373051805155],
 'test_acc': [0.921875, 0.9241071428571429, 0.9285714285714286]}

In [25]:
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

In [27]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.models.efficientnet as efficientnet

# Step 1: Load the pre-trained EfficientNetV2_M model without weights
model = models.efficientnet_v2_m(weights=None)

# Step 2: Freeze the feature extraction layers
for param in model.features.parameters():
    param.requires_grad = False

# Step 3: Modify the classifier for your specific problem (with custom number of classes)
num_classes = len(class_names)  # Assuming you have defined `class_names`
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=num_classes, bias=True)
)

# Move the model to the device (e.g., GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# (Optional) Load weights if you have a trained model already
# Load saved weights from the .pth file (if you want to fine-tune or continue training)
# model.load_state_dict(torch.load('model_final_pakka_weights.pth', map_location=device))

# Step 4: Save the model after fine-tuning or training
torch.save(model.state_dict(), 'new_trained_model_msa.pth')

print('Model saved successfully!')


Model saved successfully!


In [7]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.5.0
torchvision version: 0.20.0


In [8]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular.going_modular import data_setup, engine
except:
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [10]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [11]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [13]:
class_names = ['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs']

In [14]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

In [9]:
torch.save(model.state_dict(), "model_final_pakka_weights.pth")

NameError: name 'model' is not defined

In [29]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=4,
              bias=True).to(device))

model.load_state_dict(torch.load('model_weights.pth'))


/var/folders/4x/9705y5ws5p31tnxd9bmg3ll00000gn/T/ipykernel_20558/4252826138.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_weig

<All keys matched successfully>

In [30]:
preprocess = transforms.Compose([
    transforms.Resize(256),    
    transforms.CenterCrop(224),  
    transforms.ToTensor(),      
    transforms.Normalize(        
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])


In [38]:
from PIL import Image
img_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_lsd_jpgs/0004.jpg'  # Replace with your image path
image = Image.open(img_path).convert('RGB')
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  


In [39]:
input_batch = input_batch.to(device)


In [40]:
model.eval()

with torch.no_grad():
    output = model(input_batch)  
_, predicted_idx = torch.max(output, 1)


In [41]:
predicted_class = class_names[predicted_idx.item()]

print(f'Predicted class: {predicted_class}')


Predicted class: processed_lsd_jpgs


In [25]:
class_names = ['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs']

In [22]:
from PIL import Image
import torch
from torchvision import transforms

def predict_image_class(image_path, model, class_names, device='cpu'):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)
    
    input_batch = input_batch.to(device)
    
    model.eval()
    
    with torch.no_grad():
        output = model(input_batch)
        
        _, predicted_idx = torch.max(output, 1)
    
    predicted_class = class_names[predicted_idx.item()]
    
    return predicted_class


In [43]:
import os
import random
import torch
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),    
    transforms.CenterCrop(224),  
    transforms.ToTensor(),      
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

def predict_random_image_from_directory(model, class_names, directory, device, num_images=1):
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')
    images = [img for img in os.listdir(directory) if img.endswith(image_extensions)]
    
    if len(images) == 0:
        print(f"No images found in directory: {directory}")
        return
    
    selected_images = random.sample(images, min(num_images, len(images)))

    for img_file in selected_images:
        img_path = os.path.join(directory, img_file)
        
        image = Image.open(img_path).convert('RGB')
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0).to(device)  

        model.eval()
        
        with torch.no_grad():
            output = model(input_batch)
            _, predicted_idx = torch.max(output, 1)
            predicted_class = class_names[predicted_idx.item()]

        print(f"Image: {img_file} -> Predicted class: {predicted_class}")

directory1 = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_lsd_jpgs'  
directory2 = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_osf_jpgs'  
directory3 = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_spider_jpgs'  
directory4 = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_tseg_jpgs'  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

predict_random_image_from_directory(model, class_names, directory1, device, num_images=20)  
predict_random_image_from_directory(model, class_names, directory2, device, num_images=20)  
predict_random_image_from_directory(model, class_names, directory3, device, num_images=20)  
predict_random_image_from_directory(model, class_names, directory4, device, num_images=20)  



Image: 0326.jpg -> Predicted class: processed_lsd_jpgs
Image: 0399.jpg -> Predicted class: processed_lsd_jpgs
Image: 0378.jpg -> Predicted class: processed_lsd_jpgs
Image: 0372.jpg -> Predicted class: processed_lsd_jpgs
Image: 0075.jpg -> Predicted class: processed_lsd_jpgs
Image: 0451.jpg -> Predicted class: processed_lsd_jpgs
Image: 0444.jpg -> Predicted class: processed_lsd_jpgs
Image: 0244.jpg -> Predicted class: processed_lsd_jpgs
Image: 0080.jpg -> Predicted class: processed_lsd_jpgs
Image: 0325.jpg -> Predicted class: processed_lsd_jpgs
Image: 0558.jpg -> Predicted class: processed_lsd_jpgs
Image: 0164.jpg -> Predicted class: processed_lsd_jpgs
Image: 0332.jpg -> Predicted class: processed_lsd_jpgs
Image: 0020.jpg -> Predicted class: processed_lsd_jpgs
Image: 0208.jpg -> Predicted class: processed_lsd_jpgs
Image: 0184.jpg -> Predicted class: processed_lsd_jpgs
Image: 0523.jpg -> Predicted class: processed_lsd_jpgs
Image: 0194.jpg -> Predicted class: processed_lsd_jpgs
Image: 002

In [23]:
predict_image_class(img_path, model, class_names, device='cpu')

'processed_spider_jpgs'

In [44]:
import os
import random
from PIL import Image
import torch
from torchvision import transforms
import torch.nn.functional as F

def predict_image_class2(image_path, model, class_names, device='cpu', confidence_threshold=50):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    tensor = preprocess(image)
    batch = tensor.unsqueeze(0)
    
    batch = batch.to(device)
    model.eval()
    
    with torch.no_grad():
        output = model(batch)
        probabilities = F.softmax(output, dim=1)
        
        _, predicted_idx = torch.max(output, 1)
        confidence = probabilities[0][predicted_idx].item() * 100  
    
    return predicted_class, confidence

def predict_images_in_directory(directory_path, model, class_names, device='cpu', confidence_threshold=50):
    predictions = {}
    
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    selected_files = random.sample(image_files, min(20, len(image_files)))
    
    for filename in selected_files:
        image_path = os.path.join(directory_path, filename)
        predicted_class, confidence = predict_image_class2(image_path, model, class_names, device, confidence_threshold)
        predictions[filename] = {'class': predicted_class, 'confidence': confidence}
    
    return predictions

directory_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_lsd_jpgs'
predictions = predict_images_in_directory(directory_path, model, class_names, device='cpu')

for image_name, prediction in predictions.items():
    print(f"Image: {image_name}, Class: {prediction['class']}, Confidence: {prediction['confidence']:.2f}%")


Image: 0149.jpg, Class: processed_lsd_jpgs, Confidence: 98.22%
Image: 0104.jpg, Class: processed_lsd_jpgs, Confidence: 96.48%
Image: 0269.jpg, Class: processed_lsd_jpgs, Confidence: 99.85%
Image: 0379.jpg, Class: processed_lsd_jpgs, Confidence: 71.18%
Image: 0133.jpg, Class: processed_lsd_jpgs, Confidence: 92.17%
Image: 0390.jpg, Class: processed_lsd_jpgs, Confidence: 97.41%
Image: 0212.jpg, Class: processed_lsd_jpgs, Confidence: 97.63%
Image: 0061.jpg, Class: processed_lsd_jpgs, Confidence: 97.07%
Image: 0414.jpg, Class: processed_lsd_jpgs, Confidence: 65.20%
Image: 0297.jpg, Class: processed_lsd_jpgs, Confidence: 89.97%
Image: 0307.jpg, Class: processed_lsd_jpgs, Confidence: 96.45%
Image: 0427.jpg, Class: processed_lsd_jpgs, Confidence: 97.38%
Image: 0546.jpg, Class: processed_lsd_jpgs, Confidence: 95.05%
Image: 0367.jpg, Class: processed_lsd_jpgs, Confidence: 86.93%
Image: 0121.jpg, Class: processed_lsd_jpgs, Confidence: 94.88%
Image: 0220.jpg, Class: processed_lsd_jpgs, Confidence: